In [1]:
# Import modules
import ee
import geemap
import geopandas as gpd
import numpy as np

In [2]:
# Initialize Earth Engine
try:
    ee.Initialize()
except ee.EEException:
    ee.Authenticate()

In [7]:
# Open source table as a dataframe
country_gpkg = r"data\zambia_boundary.gpkg"
country_gdf = gpd.read_file(country_gpkg)

# Preview the data
# country_gdf.head()

In [ ]:
# Get images for rainy season (Nov to Mar) only
def get_rain_season_images(rainfall_dataset, start_year, end_year):
    # Create empty image collection
    merged_image_collection = ee.ImageCollection([])
    
    for year in range(start_year, end_year):
        # Nov to Dec of current year
        start1 = ee.Date(f'{year}-11-01')
        end1 = ee.Date(f'{year}-12-31')
        season_part1 = rainfall_dataset.filterDate(start1, end1)

        # Jan to Mar of following year
        start2 = ee.Date(f'{year + 1}-01-01')
        end2 = ee.Date(f'{year + 1}-03-31')
        season_part2 = rainfall_dataset.filterDate(start2, end2)

        # # Merge images into single collection
        # image_collections.append(season_part1)
        # image_collections.append(season_part2)

        merged_year_collection = season_part1.merge(season_part2)
        merged_image_collection = merged_image_collection.merge(merged_year_collection)

    # combined_image_collection = season_part1.merge(season_part2)

    return merged_image_collection

In [96]:
# rainfall_dataset = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD") \
#             .select('precipitation') \
#             .filterDate('1920-11-01', '1920-11-02')

# rainfall_dataset

In [115]:
# Convert county from gdf to ee
country_ee = geemap.gdf_to_ee(country_gdf)

# Import rainfall dataset from GEE
rainfall_dataset = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD") \
            .select('precipitation') 


# rainfall_dataset
# Get images for Nov 2019 to Mar 2025
seasonal_rainfall_dataset = get_rain_season_images(rainfall_dataset, 2019, 2025 )

# Clip images to AOI and compute the mean
avg_rainfall = seasonal_rainfall_dataset.mean() \
                .clip(country_ee)

seasonal_rainfall_dataset

In [112]:
# Rainfall visualisastion parameters
vis_params = {'palette': ['white', 'lightblue', 'blue', 'darkblue'], 'min': 5, 'max': 60}

# Visualise layers
m = geemap.Map()
m.zoom_to_gdf(country_gdf)
m.add_layer(country_ee, name= 'Zambia')
m.add_ee_layer(avg_rainfall, vis_params= vis_params, name= "Rainfall")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…